It is a study note over http://interactivepython.org/runestone/static/pythonds/index.html by Bradley N. Miller, David L. Ranum  which is licensed under a Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License.

# Hashing

Shared and adapted by: YinTaiChen

__Hashing__ is a data structure that can be searched in O(1) time.

If every item is where it should be, then the search can use a single comparison to discover the presence of an item.

A __hash table__ is a collection of items which are stored in such a way as to make it easy to find them later.

Each position of the hash table, often called a __slot__, can hold an item and is named by an integer value starting at 0.

We can implement a hash table by using a list with each element initialized to the special Python value <font color=red>None</font>.

![title](http://interactivepython.org/runestone/static/pythonds/_images/hashtable.png)

The mapping between an item and the slot where that item belongs in the hash table is called the __hash function__.

Assume that we have the set of integer items 54, 26, 93, 17, 77, and 31.

Our first hash function, sometimes referred to as the "remainder method", simply takes and item and divides it by the table size, returning the remainder.

| Item | Hash Value |
|------|------------|
| 54 | 10 |
| 26 | 4 |
| 93 | 5 |
| 17 | 6 |
| 77 | 0 |
| 31 | 9 |

Once the hash values have been computed, we can insert each item into the hash table at the designated position.

Now when we want to search for an item, we simply use the hash function to compute the slot name for the item and then check the hash table to see if it is present.

This searching operation is O(1).

Notice that this technique is going to work only if each item maps to a unique location in the hash table.

Two or more items need to be in the same slot is referred to as a __collision__.

## Hash Functions

Given a collection of items, a hash function that maps each item into a unique slot is referred to as a __perfect hash function__.

Luckily, we do not need the hash function to be perfect to still gain performance efficiency.

Our goal is to create a hash function that minimizes the number of collisions, is easy to compute, and evenly distributes the items in the hash table.

There are a number of common ways to extend the simple remainder method:

* The __folding method__ for constructing hash functions begins by dividing the item into equal-size pieces.

* The __mid-square method__ squares the item and then extracts some portion of the resulting digits.

We can also create hash functions for character-based items such as strings.

The word "cat" can be thought of as a sequence of ordinal values.

In [1]:
for character in "cat":
    print(character, ": ", ord(character))

c :  99
a :  97
t :  116


We can then take these three ordinal values, add them up, and use the remainder method to get a hash value.

In [2]:
def hash(astring, tablesize):
    sum = 0
    for pos in range(len(astring)):
        sum = sum + ord(astring[pos])
        
    return sum%tablesize

In [3]:
hash("cat", 11)

4

![title](http://interactivepython.org/runestone/static/pythonds/_images/stringhash.png)

It is interesting to note that when using this hash function, anagrams will always be given the same hash value.

To remedy this, we could use the position of the character as a weight.

In [4]:
def weighted_hash(astring, tablesize):
    sum = 0
    for pos in range(len(astring)):
        sum += ord(astring[pos]) * (pos + 1)
        
    return sum % tablesize

In [5]:
weighted_hash("cat", 11)

3

![title](http://interactivepython.org/runestone/static/pythonds/_images/stringhash2.png)

The important thing to remember is that the hash function has to be efficient so that it does not become the dominant part of the storage and search process.

## Collision Resolution

When two items hash to the smae slot, we must have a systematic method for placing the second item in the hash table.

This process is called __collision resolution__.

A simple way to do this is to start at the original hash value position and then move in a sequential manner through the slots until we encounter the first slot that is empty.

This collision resolution process is referred to as __open addressing__.

By systematically visiting each slot one at a time, we are performing an open addressing technique called __linear probing__.

Once we have built a hash table using open addressing and linear probing, it is essential that we utilize the same methods to search for items.

A disadvantage to linear probing is the tendency for __clustering__.

This means that if many collisions occur at the same hash value, a number of surrounding slots will be filled by the linear probing resolution.

One way to deal with clustering is to extend the linear probing technique so that instead of looking sequentially for the next open slot, we skip slots.

The general name for this process of looking for another slot after a collision is __rehashing__.

In general, _rehash(pos)_ = _(pos + skip)%sizeoftable_.

It is important to note that the size of the "skip" must be such that all the slots in the table will eventually be visited.

To ensure this, it is often suggested that the table size be a prime number.

A variation of the linear probing idea is called __quadratic probing__.

Instead of using a constant "skip" value, we use a rehash function that increments the hash value by 1, 3, 5, 7, 9, and so on.

An alternative method for handling the collision problem is to allow each slot to hold a reference to a collection (or chain) of items.

__Chaining__ allows many items to exist at the same location in the hash table.

![title](http://interactivepython.org/runestone/static/pythonds/_images/chaining.png)

## Implementing the <font color=red>__Map__</font> Abstract Data Type

In python, a dictionary is an associative data type where you can store key-data pairs.

We often refer to this idea as a __map__.

The map abstract data type is defined as follows.

* <font color=red>__Map()__</font> Create a new, empty map. It returns an empty map collection.

* <font color=red>__put(key,val)__</font> Add a new key-value pair to the map. If the key is already in the map then replace the old value with the new value.

* <font color=red>__get(key)__</font> Given a key, return the value stored in the map or <font color=red>None</font> otherwise.

* <font color=red>__del__</font> Delete the key-value pair from the map using a statement of the form <font color=red>del map[key]</font>.

* <font color=red>__len()__</font> Return the number of key-value pairs stored in the map.

* <font color=red>__in__</font> Return <font color=red>True</font> for a statement of the form <font color=red>key in map</font>. If the given key is in the map, <font color=red>False</font> otherwise.

We use two lists to create a <font color=red>HashTable</font> class that implements the Map abstract data type.

In [6]:
class HashTable:
    def __init__(self):
        self.size = 11
        self.slots = [None] * self.size
        self.data = [None] * self.size
        
    def put(self, key, data):
        hashvalue = self.hashfunction(key, len(self.slots))
        
        if self.slots[hashvalue] == None:
            self.slots[hashvalue] = key
            self.data[hashvalue] = data
        else:
            if self.slots[hashvalue] == key:
                self.data[hashvalue] = data
            else:
                nextslot = self.rehash(hashvalue, len(self.slots))
                while self.slots[nextslot] != None and self.slots[nextslot] != key:
                    nextslot = self.rehash(nextslot, len(self.slots))
                
                if self.slots[nextslot] == None:
                    self.slots[nextslot] = key
                    self.data[nextslot] = data
                else:
                    self.data[nextslot] = data
                    
    def hashfunction(self, key, size):
        return key % size
    
    def rehash(self, oldhash, size):
        return (oldhash + 1) % size
    
    def get(self, key):
        startslot = self.hashfunction(key, len(self.slots))
        
        data = None
        stop = False
        found = False
        position = startslot
        
        while self.slots[position] != None and not found and not stop:
            if self.slots[position] == key:
                found = True
                data = self.data[position]
            else:
                position = self.rehash(position, len(self.slots))
                if position == startslot:
                    stop = True
        return data
    
    def __getitem__(self, key):
        return self.get(key)
    
    def __setitem__(self, key, data):
        self.put(key, data)

In [7]:
H = HashTable()

In [8]:
H[54] = "cat"
H[26] = "dog"
H[93] = "lion"
H[17] = "tiger"
H[77] = "bird"
H[31] = "cow"
H[44] = "goat"
H[55] = "pig"
H[20] = "chicken"

In [9]:
H.slots

[77, 44, 55, 20, 26, 93, 17, None, None, 31, 54]

In [10]:
H.data

['bird',
 'goat',
 'pig',
 'chicken',
 'dog',
 'lion',
 'tiger',
 None,
 None,
 'cow',
 'cat']

In [11]:
H[20]

'chicken'

In [12]:
H[20] = 'duck'

In [13]:
H[20]

'duck'